In [1]:
from pyspark.sql import SparkSession 

In [2]:
spark = (SparkSession
 .builder
 .appName('spark')
 .enableHiveSupport()
 .getOrCreate())

1) Сгенерировать DataFrame из трёх колонок (row_id, discipline, season) - олимпийские дисциплины по сезонам.
  row_id - число порядкового номера строки;
  discipline - наименование олимпийский дисциплины на английском (полностью маленькими
буквами);
  season - сезон дисциплины (summer / winter);
*Укажите не менее чем по 5 дисциплин для каждого сезона.
Сохраните DataFrame в csv-файл, разделитель колонок табуляция, первая строка должна содержать название колонок.
Данные должны быть сохранены в виде 1 csv-файла а не множества маленьких.

In [3]:
data = [
    (1, "athletics", "summer"),
    (2, "swimming", "summer"),
    (3, "rowing", "summer"),
    (4, "sailing", "summer"),
    (5, "tennis", "summer"),
    (6, "triathlon", "winter"),
    (7, "hockey", "winter"),
    (8, "judo", "winter"),
    (9, "basketball", "winter"),
    (10, "wrestling", "winter")
]
schema = "row_id INT, discipline STRING, season STRING"
df = spark.createDataFrame(data, schema)
df.show()
df.coalesce(1).write.csv("data.csv", sep="\t", header=True, mode="overwrite")

+------+----------+------+
|row_id|discipline|season|
+------+----------+------+
|     1| athletics|summer|
|     2|  swimming|summer|
|     3|    rowing|summer|
|     4|   sailing|summer|
|     5|    tennis|summer|
|     6| triathlon|winter|
|     7|    hockey|winter|
|     8|      judo|winter|
|     9|basketball|winter|
|    10| wrestling|winter|
+------+----------+------+



2) Прочитайте исходный файл "Athletes.csv".
Посчитайте в разрезе дисциплин сколько всего спортсменов в каждой из дисциплин принимало участие.
Результат сохраните в формате parquet.

In [4]:
df = spark.read.csv("Athletes.csv", sep=";", header=True, inferSchema=True)
df.show()

df.createOrReplaceTempView("tmp_athletes")

df = spark.sql("""
SELECT discipline, count(*) as count_athletes_in_discipline 
FROM tmp_athletes 
GROUP BY discipline
""")
df.show()

df.coalesce(1).write.parquet("sportsman.parquet", mode="overwrite")

+--------------------+--------------------+-------------------+
|                Name|                 NOC|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

3) Прочитайте исходный файл "Athletes.csv".
Посчитайте в разрезе дисциплин сколько всего спортсменов в каждой из дисциплин принимало участие.
Получившийся результат нужно объединить с сгенерированным вами DataFrame из 1-го задания и в итоге вывести количество участников, только по тем дисциплинам, что есть в вашем сгенерированном DataFrame.
Результат сохраните в формате parquet.

In [5]:
athletes_df = spark.read.csv("Athletes.csv", sep=";", header=True, inferSchema=True)
athletes_df.show()

athletes_df.createOrReplaceTempView("tmp_athletes")

filtered_df = spark.read.csv("data.csv", sep="\t", header=True, inferSchema=True)
filtered_df.show()
filtered_df.createOrReplaceTempView("tmp_filtered")

athletes_df = spark.sql("""
SELECT tmp_athletes.discipline, count(*) as count_athletes_in_discipline 
FROM tmp_athletes JOIN tmp_filtered ON lower(tmp_athletes.Discipline) = tmp_filtered.discipline 
GROUP BY tmp_athletes.discipline
""")
athletes_df.show()

athletes_df.coalesce(1).write.parquet("athletes_filtered.parquet", mode="overwrite")

+--------------------+--------------------+-------------------+
|                Name|                 NOC|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .